In [ ]:
import requests
import pandas as pd
import io
import csv

In [ ]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
input_path = ''
files = {'addressFile': (input_path, open(input_path, 'rb'), 'text/csv')}
payload = {'benchmark':'Public_AR_Current', 'vintage':'Current_Current'}
print('Posting requests...')
s = requests.post(url, files=files, data=payload)
print(type(s))

In [ ]:

df = pd.read_csv(io.StringIO(s.text), sep=',', header=None, quoting=csv.QUOTE_ALL, on_bad_lines='skip')
pd.DataFrame.head(df)

In [ ]:
df.columns = ['id', 'address_in', 'match_indicator', 'match_type', 'address_out', 'long_lat', 'tiger_edge', 'street_side', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK']
pd.DataFrame(df)

In [ ]:
df[['long', 'lat']] = df['long_lat'].str.split(',', expand=True)
pd.DataFrame(df['id', 'long', 'lat'])

In [ ]:
pd.DataFrame.head(df[['id', 'long', 'lat']])

In [ ]:
df.to_csv('out.csv')